In [1]:
import os
import mlflow

In [18]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet,LinearRegression
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

In [4]:
import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [6]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    mape = mean_absolute_percentage_error(actual, pred)
    return rmse, mae, r2, mape

In [7]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [21]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )
    
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    
    alpha = 0.5
    l1_ratio = 0.5
    
    with mlflow.start_run():
        lr = LinearRegression()
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2, mape) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        print("  MAPE: %s" % mape)    


        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mape", mape)
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
        else:
            mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.6603811224461619
  MAE: 0.5073644199888924
  R2: 0.3820931219110161
  MAPE: 9.198541179272663


In [22]:
!mlflow ui

[2021-04-21 16:48:55 +0530] [1684] [INFO] Starting gunicorn 20.1.0
[2021-04-21 16:48:55 +0530] [1684] [INFO] Listening at: http://127.0.0.1:5000 (1684)
[2021-04-21 16:48:55 +0530] [1684] [INFO] Using worker: sync
[2021-04-21 16:48:55 +0530] [1687] [INFO] Booting worker with pid: 1687
Traceback (most recent call last):
  File "/home/tiger0xxx/miniconda3/envs/wq/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 697, in _list_run_infos
    run_info = self._get_run_info_from_dir(r_dir)
  File "/home/tiger0xxx/miniconda3/envs/wq/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 527, in _get_run_info_from_dir
    meta = read_yaml(run_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/tiger0xxx/miniconda3/envs/wq/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 170, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file './mlruns/0/.ipynb_checkpoints/m